In [2]:
import statsmodels.formula.api as smf
import wooldridge as woo
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import statsmodels.api as sm
import scipy.stats as stats
import patsy as pt


# Lecture 6

## Nathan Kunz

In [3]:
data = woo.data("gpa3")

### AIC

$$ AIC = ln(\frac{SSE}{N})+\frac{2K}{N}$$

In [4]:
# Fit the model
model = smf.ols('cumgpa ~ sat +hsperc +tothrs +female +black + white', data)
results = model.fit()

In [5]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 cumgpa   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     38.31
Date:                Fri, 04 Nov 2022   Prob (F-statistic):           1.73e-40
Time:                        16:00:13   Log-Likelihood:                -929.74
No. Observations:                 732   AIC:                             1873.
Df Residuals:                     725   BIC:                             1906.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8791      0.298      2.952      0.003       0.294       1.464
sat            0.0009      0.000      3.743      0.000       0.000       0.001
hsperc        -0.0056      0.002     -3.463      0.001      -0.009      -0.002
tothrs         0.0121      0.001     12.941      0.000       0.010       0.014
female         0.1667      0.077      2.164      0.031       0.015       0.318
black         -0.0261      0.192     -0.136      0.892      -0.403       0.351
white          0.0143      0.184      0.078      0.938      -0.347       0.375
==============================================================================
Omnibus:                       15.717   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.426
Skew:                           0.088   Prob(JB):                     6.72e-07
Kurtosis:                       3.949   Cond. No.                     1.01e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:
# manual AIC using the formula
def AIC(model, y):
    
    # get SSE
    SSE = ((model.resid)**2).sum()
    
    # Get K and N
    k = len(results.params)
    N = len(results.fittedvalues)
    
    # Calculate and return AIC
    return np.log(SSE/N) + ((2*k)/N)

AIC(results, data.cumgpa)

-0.2784906693074793

In [7]:
results.aic

1873.470842678566

In [8]:
# -2*llf+2(df_model + 1)
-2*results.llf+2*len(results.params)

1873.470842678566

In [9]:
-2*results.llf+14

1873.470842678566

### BIC

Prefers smaller models than AIC

$$ AIC = \ln(\frac{SSE}{N})+\frac{K\ln(N)}{N}$$

In [10]:
def BIC(model, y):
    
    # get SSE
    SSE = ((model.resid)**2).sum()
    
    # Get K and N
    k = len(results.params)
    N = len(results.fittedvalues)
    
    # Calculate and return AIC
    return np.log(SSE/N) + ((k*np.log(N))/N)

BIC(results, data.cumgpa)

-0.2345419485455542

In [11]:
results.bic

1905.6413062762952

In [12]:
# -2*llf+ln(n)*(df_model + 1)
-2*results.llf+np.log(len(results.fittedvalues))*len(results.params)

1905.6413062762952

We prefer models with the smallest AIC and BIC. We are not concerned about the absolute value.

## Heteroscedasticity

### Spread-Level Plots

In [13]:
def spread_level(model, data):
    df_copy = data.copy()
    
    # Get the studentized residuals
    df_copy["Absolute_Studentized_Residuals"] = (np.abs(model.get_influence().resid_studentized))
    df_copy["Fitted_Values"] = (model.fittedvalues)
    
    # run regression to get slope of fitted vs resid, rlm is a robust linear model used by R
    slreg = smf.rlm("np.log(Absolute_Studentized_Residuals) ~ np.log(Fitted_Values)", df_copy).fit()
    slope = slreg.params[1]
    
    # plot values
    fig, ax = plt.subplots(figsize = (10, 6))
    ax.set_title("Fitted Values vs Studentized Residuals")
    sns.regplot(x = "Fitted_Values", y = "Absolute_Studentized_Residuals", data = df_copy, lowess = True, ax = ax)
    ax.plot(df_copy.Fitted_Values.values, np.exp(slreg.fittedvalues).values)
    
    # Set to the logarithmic scale
    ax.set_yscale('log')
    ax.set_xscale('log')
    
    # convert froms scientific notation to scalar notation
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.xaxis.set_major_formatter(ScalarFormatter())
    
    # Resolve overlapping label bug
    ax.minorticks_off()
    
    # Set tick labels automatically
    ax.set_xticks(np.linspace(df_copy["Fitted_Values"].min(),df_copy["Fitted_Values"].max(), 6))
    ax.set_yticks(np.linspace(df_copy["Absolute_Studentized_Residuals"].min(),
                              df_copy["Absolute_Studentized_Residuals"].max(), 6))
    
    ax.grid()
    
    # return a suggested power transform of your y-variable that may correct heteroscedastcity
    # The transform is just one minus the slope of the reegression line of your fitted values vs residuals
    print("Suggested Power Transformation:", 1-slope)

In [14]:
foodata = pd.read_csv("food.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'food.csv'

In [ ]:
results2 = smf.ols('food_exp~income', foodata).fit()
results2.summary()

In [ ]:
results2 = smf.ols('food_exp~income', foodata).fit()
spread_level(results2, foodata)

In [ ]:
# Fit the model
model = smf.ols('cumgpa ~ sat +hsperc +tothrs +female +black + white', data)
results = model.fit()
spread_level(results, data)

In [ ]:
baseball = pd.read_csv('baseball.csv').dropna()
results3 = smf.ols('sal87 ~ years + hits + runs + homeruns', baseball).fit()

In [ ]:
spread_level(results3, baseball)

### Tests for Nonconstant Error Variance

### Bruesch-Pagan Test

In [ ]:
# pull out squared residuals
data["res2"] = results.resid**2

# try to predict the squared residuals using a linear combination of our variables
aux_reg = smf.ols('res2 ~ sat +hsperc +tothrs +female +black + white', data).fit()

# Get the regression f-statistic (f-test version)
f = aux_reg.fvalue
fp = aux_reg.f_pvalue

print("The F-Statistic for the Auxiliary Regression is: "+ str(f) +" and the P-Value is: "+ str(fp))

Therefore we reject the null hypothesis that $\delta_1 =\delta_2=...= \delta_k = 0$ and conclude heteroscedasticity is present in the sample. 

In [ ]:
y, X = pt.dmatrices('cumgpa ~ sat +hsperc +tothrs +female +black + white', data,
                   return_type = 'dataframe')

# Takes in the residuals and our design matrix as arguments
# Order is Lm Test statistic, LM P-value, F-stat, F-Pvalue
sm.stats.diagnostic.het_breuschpagan(results.resid, X)

In [ ]:
# LM test statsitic is just n*R2 from the aux regression
LM = len(data)*aux_reg.rsquared

k = results.df_model

In [ ]:
# sf is just 1- cdf (called the survival function)
stats.chi2(k).sf(LM)

### White Test

The white test procedure is equivalent to the BP test, except that 2nd order terms and interactions are added to teh auxiliary regression. Note that this will eat up degrees of freedom for this implementation of the test. 

In [ ]:
# Order is Lm Test statistic, LM P-value, F-stat, F-Pvalue
sm.stats.diagnostic.het_white(results.resid, X)

### Goldfeld-Quandt Test

In [ ]:
# I need to provide a split point to the software
# Sprt values in ascending order and reset the index to number from 1 to len(data)
sortedv = data.sort_values(by = "black").copy().reset_index()

# This returns the first index that contains a one
splt = sortedv.black.argmax()

# run regression
gq_reg = smf.ols('cumgpa ~ sat +hsperc +tothrs +female +black + white', sortedv).fit()

In [ ]:
# get the data for dependent and independent variables
# these are numpy arrays instead of dataframes
y = gq_reg.model.endog
X = gq_reg.model.exog

# Order is f-stat, pvalue, hypothesis
sm.stats.diagnostic.het_goldfeldquandt(y, X, idx = 5, alternative = 'increasing', split= splt)

In [ ]:
# get the data for dependent and independent
y = gq_reg.model.endog
X = gq_reg.model.exog

# Order is f-stat, pvalue, hypothesis
sm.stats.diagnostic.het_goldfeldquandt(y, X, idx = 5, alternative = 'two-sided', split= splt)

In [ ]:
# manual implementation
data1 = data[data.black == 1]
data0 = data[data.black == 0]

# run regs on different groups
reg1 = smf.ols('cumgpa ~ sat +hsperc +tothrs +female +black + white', data1).fit()
reg0 = smf.ols('cumgpa ~ sat +hsperc +tothrs +female +black + white', data0).fit()

# pull out the residuals of each regression
df1 = reg1.df_resid
df0 = reg0.df_resid

# Get the variance of each regression
sig1squared = reg1.scale
sig0squared = reg0.scale

fstat = sig1squared/sig0squared

# calculate critical calue for right side test
stats.f.ppf(.95, df1, df0)

In [ ]:
fstat

$F = \frac{\hat{\sigma}_1^2}{\hat{\sigma}_0^2}$, where $H_0: \hat{\sigma}_1^2 \leq \hat{\sigma}_0^2$ and $H_a: \hat{\sigma}_1^2 > \hat{\sigma}_0^2$

In [ ]:
y, X = pt.dmatrices('cumgpa ~ sat +hsperc +tothrs +female +black + white', sortedv,
                   return_type = 'dataframe')

In [ ]:
X.iloc[:,5]

We fail to reject the null hypothesis

### Robust Standard Errors

In [ ]:
robust_reg = smf.ols('cumgpa ~ sat +hsperc +tothrs +female +black + white', data).fit(cov_type = 'HC0')
robust_reg.summary()

In [ ]:
results.summary()

In [ ]:
# pull out squared residuals
data["res2"] = robust_reg.resid**2

# try to predict the squared residuals using a linear combination of our variables
aux_reg = smf.ols('res2 ~ sat +hsperc +tothrs +female +black + white', data).fit()

# Get the regression f-statistic (f-test version)
f = aux_reg.fvalue
fp = aux_reg.f_pvalue

print("The F-Statistic for the Auxiliary Regression is: "+ str(f) +" and the P-Value is: "+ str(fp))

### Weighted Least Squares

Right now we have a regression model where the errors are asumed to be heteroscedastic:

$$cumgpa_i = \beta_0+ \beta_1 sat +\beta_2 hsperc +\beta_3 tothrs +\beta_4 female +\beta_5 lack + \beta_6 white + e_i$$, $$var(e_i) = \sigma^2_i$$

We would like to know the variance for each pbservation/quantile. This isn't possible so instead we can make an assumption about how the variance of our errors depends on one or several of the regressors. Making this assumption gives us the generalized least square (GLS) estimator. This involves a linear relationship between variance and regressor $x_i$. An example of such a relationship would be:
$$var(e_i)=\sigma^2_i = \sigma^2x_i$$

where the varaince for observation i is a function of some consatnt and the current value of x (so variance would tend to increase as x increases). Then if we can correctly guess the structure of the error term, we, for example, divide our model by $\sqrt(x_i)$ to yieldf a BLUE estimator

In [ ]:
# maybe I believe the heteroscedastic relationship is due to sat
w = 1/data.sat

# run a weighted regression and provide weights
# note we can use WLS and robust standrad errors
wls_known = smf.wls('cumgpa ~ sat +hsperc +tothrs +female +black + white', weights = w, data = data).fit()
wls_known.summary()

### Feasible GLS

Often times we have no diea what the appropriate weights for a model might be. FGLS is a proceudre for estimatinng the proper weights to be included in wls. Here we assume that the variance of the error terms is a function of some variables. 

In [ ]:
results2.summary()

In [ ]:
foodata["ehatsq"] = results2.resid**2

# estimate weights
w_est = smf.ols('np.log(ehatsq) ~ income', data = foodata).fit()

vari = np.exp(w_est.fittedvalues) #estimated variances
w = 1/vari**2

fgls =smf.wls('food_exp ~ income', foodata, weights = w).fit()

fgls.summary()

## Missing Observations

In [ ]:
housing = pd.read_csv('bh.csv')
housing.isna().sum()

### Removal (Variable or Observation)

In [ ]:
housing.shape

In [ ]:
# Generate some random indices to craete NAs
NA_indices1 = np.random.choice(range(0,len(housing)), 100, replace = False)
NA_indices2 = np.random.choice(range(0,len(housing)), 100, replace = False)
# Add NAS to a few columns
housing.loc[NA_indices1,"crim"] = np.NaN
# Add NAS to a few columns
housing.loc[NA_indices2,"tax"] = np.NaN

In [ ]:
housing.isna().sum()

### Mean, Median, or Mode Imputation

In [ ]:
stat_copy = housing.copy()

# just fill in with your favored statistic
stat_copy["crim"] = housing.crim.fillna(housing.crim.median())
housing.crim.median()

In [ ]:
stat_copy.isna().sum()

In [ ]:
stat_copy.head(10)

### Prediction

#### KNN

https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html

In [ ]:
from sklearn.impute import KNNImputer

# knn imputer uses 5 as a default
imputer = KNNImputer(n_neighbors=5)

knn_df = pd.DataFrame(imputer.fit_transform(housing), columns = housing.columns)
knn_df.head(10)

In [ ]:
knn_df.isna().sum()

#### Recursive Partitioning

In [19]:
data2 = woo.data("hprice1")
data2

,price,assess,bdrms,lotsize,sqrft,colonial,lprice,lassess,llotsize,lsqrft
0,300.0,349.100006,4,6126.0,2438,1,5.703783,5.855359,8.720297,7.798934
1,370.0,351.500000,3,9903.0,2076,1,5.913503,5.862210,9.200593,7.638198
2,191.0,217.699997,3,5200.0,1374,0,5.252274,5.383118,8.556414,7.225482
3,195.0,231.800003,3,4600.0,1448,1,5.273000,5.445875,8.433811,7.277938
4,373.0,319.100006,4,6095.0,2514,1,5.921578,5.765504,8.715224,7.829630
...,...,...,...,...,...,...,...,...,...,...
83,295.0,318.299988,3,6056.0,1837,1,5.686975,5.762994,8.708805,7.515889
84,236.0,259.399994,3,5828.0,1715,0,5.463832,5.558371,8.670429,7.447168
85,202.5,258.100006,3,6341.0,1574,0,5.310740,5.553347,8.754792,7.361375
86,219.0,232.000000,2,6362.0,1185,0,5.389072,5.446737,8.758098,7.077498


In [21]:
model = smf.ols('price ~ lotsize + sqrft + bdrms ', data2)
results2 = model.fit()

In [22]:
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     57.46
Date:                Fri, 04 Nov 2022   Prob (F-statistic):           2.70e-20
Time:                        16:05:22   Log-Likelihood:                -482.88
No. Observations:                  88   AIC:                             973.8
Df Residuals:                      84   BIC:                             983.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -21.7703     29.475     -0.739      0.462     -80.385      36.844
lotsize        0.0021      0.001      3.220      0.002       0.001       0.003
sqrft          0.1228      0.013      9.275      0.000       0.096       0.149
bdrms         13.8525      9.010      1.537      0.128      -4.065      31.770
==============================================================================
Omnibus:                       20.398   Durbin-Watson:                   2.110
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.278
Skew:                           0.961   Prob(JB):                     9.79e-08
Kurtosis:                       5.261   Cond. No.                     6.41e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.41e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
robust_reg = smf.ols('price ~ lotsize + sqrft + bdrms ', data2).fit(cov_type = 'HC0')
robust_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     24.85
Date:                Fri, 04 Nov 2022   Prob (F-statistic):           1.33e-11
Time:                        16:28:04   Log-Likelihood:                -482.88
No. Observations:                  88   AIC:                             973.8
Df Residuals:                      84   BIC:                             983.7
Df Model:                           3                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -21.7703     36.284     -0.600      0.549     -92.886      49.346
lotsize        0.0021      0.001      1.691      0.091      -0.000       0.004
sqrft          0.1228      0.017      7.090      0.000       0.089       0.157
bdrms         13.8525      8.284      1.672      0.094      -2.383      30.088
==============================================================================
Omnibus:                       20.398   Durbin-Watson:                   2.110
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               32.278
Skew:                           0.961   Prob(JB):                     9.79e-08
Kurtosis:                       5.261   Cond. No.                     6.41e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 6.41e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
# pull out squared residuals
data["res2"] = results.resid**2

# try to predict the squared residuals using a linear combination of our variables
aux_reg = smf.ols('price ~ lotsize + sqrft + bdrms ', data2).fit()

# Get the regression f-statistic (f-test version)
f = aux_reg.fvalue
fp = aux_reg.f_pvalue

print("The F-Statistic for the Auxiliary Regression is: "+ str(f) +" and the P-Value is: "+ str(fp))

The F-Statistic for the Auxiliary Regression is: 57.46023202598372 and the P-Value is: 2.695972654702694e-20


In [28]:
data["res2"] = results.resid**2

# estimate weights
w_est = smf.ols('price ~ lotsize + sqrft + bdrms ', data2 ).fit()

vari = np.exp(w_est.fittedvalues) #estimated variances
w = 1/vari**2

fgls =smf.wls('price ~ lotsize + sqrft + bdrms ', data2, weights = w).fit()

fgls.summary()

C:\Users\HP Pavilion 13\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:776: RuntimeWarning: divide by zero encountered in log
  llf += 0.5 * np.sum(np.log(self.weights))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       1.000
Model:                            WLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.295e+20
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        16:31:27   Log-Likelihood:                   -inf
No. Observations:                  88   AIC:                               inf
Df Residuals:                      86   BIC:                               inf
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0002   2.38e-15    9.9e+10      0.000       0.000       0.000
lotsize       -0.0138   4.87e-13  -2.84e+10      0.000      -0.014      -0.014
sqrft          0.2591   2.61e-12   9.91e+10      0.000       0.259       0.259
bdrms          0.0019   1.95e-14   9.73e+10      0.000       0.002       0.002
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.000
Prob(Omnibus):                    nan   Jarque-Bera (JB):                  nan
Skew:                          -7.466   Prob(JB):                          nan
Kurtosis:                         nan   Cond. No.                     2.08e+19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.08e-174. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""